In [13]:
import numpy as np
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import requests

In [ ]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline

# Getting the Data

We are going to utilize the Open Opus API to retrieve our metadata 

In [29]:
# composer_response = requests.get("https://api.openopus.org/composer/list/pop.json")
# print(response.status_code)

ConnectionError: HTTPSConnectionPool(host='api.openopus.org', port=443): Max retries exceeded with url: /composer/list/pop.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000225B3458CA0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

This call gets a list of popular composers. Let's take a look.

In [27]:
# print(composer_response.json())

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Only run the following once to retrieve Open Opus entire dataset:

In [18]:
# response = requests.get("https://api.openopus.org/work/dump.json")
# print(response.status_code)

In [25]:
# json_data = response.json()
# with open('data.json', 'w') as f:
#     json.dump(json_data, f)

### Let's take a look at the types of meta data

### Attributes for each Composer:
- ID
- Name
- Complete Name
- Birth
- Death
- Epoch (period)
  - Medieval
  - Renaissance
  - Baroque
  - Classical
  - Early Romantic
  - Romantic
  - Late Romantic
  - 20th Century
  - Post-War
  - 21st Century
- Portrait

### Attributes for each work:
- Title
- Subtitle
- Searchterms
- Popular
- Genre:
  - Chamber
  - Keyboard
  - Orchestral
  - Stage
  - Vocal